In [1]:
import numpy as np
import json
import pandas as pd
import matplotlib as plt
import seaborn as sns
import csv
from tweepy.streaming import StreamListener
from tweepy import OAuthHandler, Stream, API, Cursor
import requests
import nltk
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize, RegexpTokenizer
from nltk.corpus import stopwords
import os
import re

In [2]:
## access token informations 
access_token1 = "1222829174802538496-AXkAccE1dWMjuEFlEYChpgOibc6SbF"
access_token_secret1 = "jtr66CvVvspsANplSBpuMEQR5iLwK2fRtzM6aDhaC1rZT"

consumer_key1 = "sL9E5QL83DVEshfttLjEEj930"
consumer_secret1 = "R9L4Ar7UWTKB4WQw7cEoybumyTInZ6pYP3cQ5GFlWYFllYAeec"

In [3]:
auth = OAuthHandler(consumer_key1, consumer_secret1)
auth.set_access_token(access_token1, access_token_secret1)
api = API(auth_handler=auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

In [4]:
class MyStreamListener(StreamListener):
    """
    Twitter listener, collects streaming tweets and output to a file
    """
    def __init__(self, api, OUTPUT_FILENAME, stop_condition=10):
        """
        initialize the stream, with num. of tweets and saving the outputfile
        """
        
        # this line is needed to import the characteristics of the streaming API
        super(MyStreamListener, self).__init__()
        
        # to-count the number of tweets collected        
        self.num_tweets = 0
        
        # save filename
        self.filename = OUTPUT_FILENAME
        
        # stop-condition
        self.stop_condition = stop_condition
        

    def on_status(self, status):
        
        """
        this function runs each time a new bunch of tweets is retrived from the streaming
        """
        
        with open(self.filename, "a+") as f:
            tweet = status._json
            
            f.write(json.dumps(tweet) + '\n')
            #self.output.append(tweet)
            self.num_tweets += 1
        
            # Stop condition        
            if self.num_tweets <= self.stop_condition:
                return True
            else:
                return False
        

    def on_error(self, status):
        """
        function useful to handle errors. It's possible to personalize it 
        depending on the way we want to handle errors
        """
        
        print(status)
        #returning False in on_error disconnects the stream
        return False

In [5]:
OUTPUT_FILENAME = "other-outputs/tweets_US_Election_2020.json"
stop_condition = 500

l = MyStreamListener(api, OUTPUT_FILENAME, stop_condition)
# here we recall the Stream Class from Tweepy to input the authentication info and our personalized listener 
stream = Stream(auth=api.auth, listener=l)


# keywords we may want decide to track 
TRACKING_KEYWORDS = ["Donald", "Trump", "Joe", "Biden", "America", "USA"]
stream.filter(
    track=TRACKING_KEYWORDS, 
    is_async=False, 
    languages = ["en"]
)

In [6]:
json_Election_file = []
for line in open(OUTPUT_FILENAME, 'r'):
    json_Election_file.append(json.loads(line))

In [7]:
dict_train={}
i=0
for train_file in json_Election_file:
    dict_train[i]=train_file
    i+=1
train = pd.DataFrame.from_dict(dict_train, orient='index')
train.reset_index(level=0, inplace=False)
train.head(10)

,created_at,id,id_str,text,source,truncated,in_reply_to_status_id,in_reply_to_status_id_str,in_reply_to_user_id,in_reply_to_user_id_str,...,entities,favorited,retweeted,filter_level,lang,timestamp_ms,possibly_sensitive,display_text_range,extended_tweet,extended_entities
0,Sat Nov 21 18:57:14 +0000 2020,1330223766018920450,1330223766018920450,RT @jimsciutto: Priorities.,"<a href=""http://twitter.com/download/iphone"" r...",False,NaN,None,NaN,None,...,"{'hashtags': [], 'urls': [], 'user_mentions': ...",False,False,low,en,1605985034838,NaN,NaN,NaN,NaN
1,Sat Nov 21 18:57:14 +0000 2020,1330223766237016066,1330223766237016066,RT @SenSanders: The American people can't affo...,"<a href=""https://mobile.twitter.com"" rel=""nofo...",False,NaN,None,NaN,None,...,"{'hashtags': [], 'urls': [], 'user_mentions': ...",False,False,low,en,1605985034890,NaN,NaN,NaN,NaN
2,Sat Nov 21 18:57:14 +0000 2020,1330223766174130176,1330223766174130176,RT @james960909: Thanks,"<a href=""http://twitter.com/download/iphone"" r...",False,NaN,None,NaN,None,...,"{'hashtags': [], 'urls': [], 'user_mentions': ...",False,False,low,en,1605985034875,NaN,NaN,NaN,NaN
3,Sat Nov 21 18:57:14 +0000 2020,1330223766379630595,1330223766379630595,RT @CNN: Twitter said it will hand control of ...,"<a href=""http://twitter.com/download/iphone"" r...",False,NaN,None,NaN,None,...,"{'hashtags': [], 'urls': [{'url': 'https://t.c...",False,False,low,en,1605985034924,False,NaN,NaN,NaN
4,Sat Nov 21 18:57:14 +0000 2020,1330223766434148368,1330223766434148368,RT @AriMelber: Donald Trump has quarantined mo...,"<a href=""https://mobile.twitter.com"" rel=""nofo...",False,NaN,None,NaN,None,...,"{'hashtags': [], 'urls': [], 'user_mentions': ...",False,False,low,en,1605985034937,NaN,NaN,NaN,NaN
5,Sat Nov 21 18:57:14 +0000 2020,1330223766505484290,1330223766505484290,RT @ReviveTheUSnow: @anthony93180 @newsbreakAp...,"<a href=""http://twitter.com/download/iphone"" r...",False,NaN,None,NaN,None,...,"{'hashtags': [], 'urls': [], 'user_mentions': ...",False,False,low,en,1605985034954,NaN,NaN,NaN,NaN
6,Sat Nov 21 18:57:14 +0000 2020,1330223766669041665,1330223766669041665,@ScullyGS @John_TheBeef Man skips boring talk ...,"<a href=""http://twitter.com/download/android"" ...",False,1.330220e+18,1330219598872973312,298745467.0,298745467,...,"{'hashtags': [], 'urls': [], 'user_mentions': ...",False,False,low,en,1605985034993,NaN,"[24, 120]",NaN,NaN
7,Sat Nov 21 18:57:14 +0000 2020,1330223766677430273,1330223766677430273,RT @varadmehta: Remember this when President D...,"<a href=""http://twitter.com/download/android"" ...",False,NaN,None,NaN,None,...,"{'hashtags': [], 'urls': [], 'user_mentions': ...",False,False,low,en,1605985034995,NaN,NaN,NaN,NaN
8,Sat Nov 21 18:57:15 +0000 2020,1330223766710984706,1330223766710984706,#COVID19 \n#coronavirus\nTrump Hotel\nDom Peri...,"<a href=""http://twitter.com/download/iphone"" r...",False,NaN,None,NaN,None,...,"{'hashtags': [{'text': 'COVID19', 'indices': [...",False,False,low,en,1605985035003,NaN,NaN,NaN,NaN
9,Sat Nov 21 18:57:14 +0000 2020,1330223766618730496,1330223766618730496,RT @ifindkarma: President* Trump has been the ...,"<a href=""http://twitter.com/download/iphone"" r...",False,NaN,None,NaN,None,...,"{'hashtags': [], 'urls': [], 'user_mentions': ...",False,False,low,en,1605985034981,NaN,NaN,NaN,NaN


In [8]:
pd.set_option('display.max_columns', None)
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1002 entries, 0 to 1001
Data columns (total 36 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   created_at                 1002 non-null   object 
 1   id                         1002 non-null   int64  
 2   id_str                     1002 non-null   object 
 3   text                       1002 non-null   object 
 4   source                     1002 non-null   object 
 5   truncated                  1002 non-null   bool   
 6   in_reply_to_status_id      123 non-null    float64
 7   in_reply_to_status_id_str  123 non-null    object 
 8   in_reply_to_user_id        129 non-null    float64
 9   in_reply_to_user_id_str    129 non-null    object 
 10  in_reply_to_screen_name    129 non-null    object 
 11  user                       1002 non-null   object 
 12  geo                        0 non-null      object 
 13  coordinates                0 non-null      objec

In [9]:
#Normalize functions:

#NO SALEN HASHTAGS NI DEL TEXT NI DE ENTITIES.MIRAR RETWEETED STATUS Y TIENE QUE DEVOLVER
def get_hashtags(data: pd.DataFrame) -> None:
    hashtag_list=[]
    for i in range(len(data)):
        hashtags=[]
        try:
            hashtags_info = data['entities']['hashtags']
        except:
            hashtags_info = data['entities'][i]['hashtags']
        for i in range (len(hashtags_info)):
            hashtags.append(hashtags_info[i]["text"])
        hashtag_list.append(hashtags)
    data['hashtags']=hashtag_list

def remove_unnecessary_columns(data: pd.DataFrame)-> pd.DataFrame:
    cols_to_remove=['id', 'display_text_range', 'in_reply_to_status_id', 
                    'in_reply_to_status_id_str', 'in_reply_to_user_id', 'in_reply_to_user_id_str', 'geo', 'coordinates', 
                    'place', 'contributors', 'is_quote_status', 'quote_count','lang', 
                    'possibly_sensitive', 'quoted_status_id','quoted_status_id_str', 'quoted_status', 
                    'quoted_status_permalink']
    
    data=data.drop(columns=cols_to_remove)
    return data

#retweeted_status only exists if the tweet has been retweeted by someone. If not, extended_tweet exists if Truncated is true.
#If both are not the case, we use directly the information given in the tweet.
def get_tweet_data(data: pd.DataFrame) -> pd.DataFrame:
    #  Tweet	| Username | Date | Hashtags | Likes | Retweets | Url
    tweets_text =[]
    tweets_username =[]
    tweets_date = []
    tweets_hashtags = []
    tweets_likes = []
    tweets_retweets = []
    tweets_url = []
    for tweet in range(len(data)):
        
        if str(data.iloc[tweet]['retweeted_status'])=='nan':
            if(data["truncated"][tweet] == False):
                tweets_text.append(data['text'][tweet])
                tweets_hashtags.append(get_hashtags(data))
            else:
                tweets_text.append(data['extended_tweet'][tweet]['full_text'])
                tweets_hashtags.append(get_hashtags(data['extended_tweet'][tweet]))
            
            tweets_date.append(data['created_at'][tweet])
            tweets_username.append(data['user'][tweet]['name'])
            tweets_url.append("https://twitter.com/i/web/status/"+ data["id_str"][tweet])
            tweets_likes.append(data['favorite_count'][tweet])
            tweets_retweets.append(data['retweet_count'][tweet])
        else:
            tweets_text.append(data['retweeted_status'][tweet]['text'])
            #We supposed it is the name and not the @name (screen_name)
            tweets_username.append(data['retweeted_status'][tweet]['user']['name']) 
            tweets_date.append(data['retweeted_status'][tweet]['created_at'])
            tweets_hashtags.append(get_hashtags((data['retweeted_status'][tweet])))
            tweets_likes.append(data['retweeted_status'][tweet]['favorite_count'])
            tweets_retweets.append(data['retweeted_status'][tweet]['retweet_count'])
            tweets_url.append("https://twitter.com/i/web/status/"+ data["id_str"][tweet])
        
    tweet_info = {'Tweet': tweets_text, 'Username': tweets_username, 'Date': tweets_date, 'Hashtags': tweets_hashtags, 
         'Likes': tweets_likes, 'Retweets': tweets_retweets, 'Url': tweets_url}

    return pd.DataFrame(data=tweet_info)

In [10]:
def decontracted(text: str) -> str:
    text = re.sub(r"won\'t", "will not", text)
    text = re.sub(r"can\'t", "can not", text)
    text = re.sub(r"n\'t", " not", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"\'s", " is", text)
    text = re.sub(r"\'d", " would", text)
    text = re.sub(r"\'ll", " will", text)
    text = re.sub(r"\'t", " not", text)
    text = re.sub(r"\'ve", " have", text)
    text = re.sub(r"\'m", " am", text)
    return text

def punctuation_removal(text: str) -> str:
    tokenizer=RegexpTokenizer(r'\w+')
    return tokenizer.tokenize(decontracted(text.lower()))

def stemming(text: list) -> list:
    ps = PorterStemmer()
    stemmed_text = []
    for word in text: 
        stemmed_text.append(ps.stem(word))
    return stemmed_text

def remove_stop_words(word_list: list) -> list:
    stop_words = set(stopwords.words('english'))
    text_without_stopwords = []
    for word in word_list:
        if not word in stop_words:
            text_without_stopwords.append(word)
    return text_without_stopwords

def text_normalization(text: str) -> str:
    text_normalized = punctuation_removal(text)
    text_stemmed  = stemming(text_normalized)
    text_without_stopwords = remove_stop_words(text_stemmed)
    return " ".join(text_without_stopwords)

In [11]:
train=remove_unnecessary_columns(train)

In [12]:
def inverted_index(data: pd.DataFrame)->dict:
    inv_index={}
    for doc_num in range(len(data['text'])):
        words=train['text'][doc_num].split()
        for word in words:
            if word in inv_index:
                inv_index[word].append(doc_num)
            else:
                inv_index[word]=[doc_num]
    return inv_index

def tf_idf(query: str, docs: pd.DataFrame) -> pd.DataFrame:
    for doc_text in len(docs):
        text=docs["Tweet"][doc_text]
            

In [13]:
database=get_tweet_data(train)
for i in range(len(train)):
    train['text'][i]=text_normalization(train['text'][i])
database

<ipython-input-13-0442b4786854>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['text'][i]=text_normalization(train['text'][i])


,Tweet,Username,Date,Hashtags,Likes,Retweets,Url
0,Priorities.,Jim Sciutto,Sat Nov 21 17:27:42 +0000 2020,None,3507,758,https://twitter.com/i/web/status/1330223766018...
1,The American people can't afford to wait any l...,Bernie Sanders,Sat Nov 21 18:55:19 +0000 2020,None,767,142,https://twitter.com/i/web/status/1330223766237...
2,Thanks,james johnson,Sat Nov 21 18:53:20 +0000 2020,None,1,1,https://twitter.com/i/web/status/1330223766174...
3,Twitter said it will hand control of the POTUS...,CNN,Sat Nov 21 10:31:03 +0000 2020,None,15038,1024,https://twitter.com/i/web/status/1330223766379...
4,Donald Trump has quarantined more strictly in ...,Ari Melber,Fri Nov 20 22:48:08 +0000 2020,None,117754,11881,https://twitter.com/i/web/status/1330223766434...
...,...,...,...,...,...,...,...
997,I hope Trump leaks Biden's phone calls to his ...,thebradfordfile™,Sat Nov 21 18:55:36 +0000 2020,None,498,137,https://twitter.com/i/web/status/1330224386327...
998,Trump haters push an elderly woman down who is...,Go Trump 🇺🇸,Sat Nov 21 18:50:33 +0000 2020,None,20,22,https://twitter.com/i/web/status/1330224386343...
999,it's a lie that Trump met with resistance in h...,Edward-Isaac Dovere,Sat Nov 21 16:10:12 +0000 2020,None,2084,573,https://twitter.com/i/web/status/1330224386511...
1000,One of the most shocking things that I heard t...,Rush Limbaugh Quotes,Fri Nov 20 00:18:06 +0000 2020,None,335,82,https://twitter.com/i/web/status/1330224386540...


In [15]:
train.head()
# get always full text by checking if truncated. If it is, then find extended_tweet or retweeted_status.

,created_at,id_str,text,source,truncated,in_reply_to_screen_name,user,retweeted_status,reply_count,retweet_count,favorite_count,entities,favorited,retweeted,filter_level,timestamp_ms,extended_tweet,extended_entities,hashtags
0,Sat Nov 21 18:57:14 +0000 2020,1330223766018920450,rt jimsciutto prioriti,"<a href=""http://twitter.com/download/iphone"" r...",False,None,"{'id': 1137795270341734401, 'id_str': '1137795...",{'created_at': 'Sat Nov 21 17:27:42 +0000 2020...,0,0,0,"{'hashtags': [], 'urls': [], 'user_mentions': ...",False,False,low,1605985034838,NaN,NaN,[]
1,Sat Nov 21 18:57:14 +0000 2020,1330223766237016066,rt sensand american peopl afford wait ani long...,"<a href=""https://mobile.twitter.com"" rel=""nofo...",False,None,"{'id': 3091417701, 'id_str': '3091417701', 'na...",{'created_at': 'Sat Nov 21 18:55:19 +0000 2020...,0,0,0,"{'hashtags': [], 'urls': [], 'user_mentions': ...",False,False,low,1605985034890,NaN,NaN,[]
2,Sat Nov 21 18:57:14 +0000 2020,1330223766174130176,rt james960909 thank,"<a href=""http://twitter.com/download/iphone"" r...",False,None,"{'id': 1230148806362173441, 'id_str': '1230148...",{'created_at': 'Sat Nov 21 18:53:20 +0000 2020...,0,0,0,"{'hashtags': [], 'urls': [], 'user_mentions': ...",False,False,low,1605985034875,NaN,NaN,[]
3,Sat Nov 21 18:57:14 +0000 2020,1330223766379630595,rt cnn twitter said hand control potu account ...,"<a href=""http://twitter.com/download/iphone"" r...",False,None,"{'id': 364969064, 'id_str': '364969064', 'name...",{'created_at': 'Sat Nov 21 10:31:03 +0000 2020...,0,0,0,"{'hashtags': [], 'urls': [{'url': 'https://t.c...",False,False,low,1605985034924,NaN,NaN,[]
4,Sat Nov 21 18:57:14 +0000 2020,1330223766434148368,rt arimelb donald trump ha quarantin strictli ...,"<a href=""https://mobile.twitter.com"" rel=""nofo...",False,None,"{'id': 1288402892819320832, 'id_str': '1288402...",{'created_at': 'Fri Nov 20 22:48:08 +0000 2020...,0,0,0,"{'hashtags': [], 'urls': [], 'user_mentions': ...",False,False,low,1605985034937,NaN,NaN,[]


In [14]:
print("Insert your query:\n")
query = input()
docs = search(query, index)    
top = 20

Insert your query:



KeyboardInterrupt: Interrupted by user

In [ ]:
os.remove(OUTPUT_FILENAME)

In [ ]:
#Quitar URLs
#Quitar emojis
#CLEAN .JSON SO IT DOES NOT SUM TWEETS IF RUNED AGAIN